In [1]:
import pandas as pd
import numpy as np
import json
import warnings
warnings.filterwarnings("ignore")
import random

In [2]:
import spacy

In [3]:
train = pd.read_csv("train.csv")
features = pd.read_csv("features.csv")
patient_notes = pd.read_csv("patient_notes.csv")

In [4]:
train.head()

,id,case_num,pn_num,feature_num,annotation,location
0,00016_000,0,16,0,['dad with recent heart attcak'],['696 724']
1,00016_001,0,16,1,"['mom with ""thyroid disease']",['668 693']
2,00016_002,0,16,2,['chest pressure'],['203 217']
3,00016_003,0,16,3,"['intermittent episodes', 'episode']","['70 91', '176 183']"
4,00016_004,0,16,4,['felt as if he were going to pass out'],['222 258']


In [5]:
features.head()

,feature_num,case_num,feature_text
0,0,0,Family-history-of-MI-OR-Family-history-of-myoc...
1,1,0,Family-history-of-thyroid-disorder
2,2,0,Chest-pressure
3,3,0,Intermittent-symptoms
4,4,0,Lightheaded


In [6]:
patient_notes.head()

,pn_num,case_num,pn_history
0,0,0,"17-year-old male, has come to the student heal..."
1,1,0,17 yo male with recurrent palpitations for the...
2,2,0,Dillon Cleveland is a 17 y.o. male patient wit...
3,3,0,a 17 yo m c/o palpitation started 3 mos ago; \...
4,4,0,17yo male with no pmh here for evaluation of p...


In [7]:
import multiprocessing

In [8]:
multiprocessing.cpu_count()

8

In [9]:
train.head()

,id,case_num,pn_num,feature_num,annotation,location
0,00016_000,0,16,0,['dad with recent heart attcak'],['696 724']
1,00016_001,0,16,1,"['mom with ""thyroid disease']",['668 693']
2,00016_002,0,16,2,['chest pressure'],['203 217']
3,00016_003,0,16,3,"['intermittent episodes', 'episode']","['70 91', '176 183']"
4,00016_004,0,16,4,['felt as if he were going to pass out'],['222 258']


In [10]:
train[(train.case_num == 0)&(train.pn_num == 16)]

,id,case_num,pn_num,feature_num,annotation,location
0,00016_000,0,16,0,['dad with recent heart attcak'],['696 724']
1,00016_001,0,16,1,"['mom with ""thyroid disease']",['668 693']
2,00016_002,0,16,2,['chest pressure'],['203 217']
3,00016_003,0,16,3,"['intermittent episodes', 'episode']","['70 91', '176 183']"
4,00016_004,0,16,4,['felt as if he were going to pass out'],['222 258']
5,00016_005,0,16,5,[],[]
6,00016_006,0,16,6,"['adderall', 'adderrall', 'adderrall']","['321 329', '404 413', '652 661']"
7,00016_007,0,16,7,[],[]
8,00016_008,0,16,8,[],[]
9,00016_009,0,16,9,"['palpitations', 'heart beating/pounding']","['26 38', '96 118']"


In [11]:
train[(train.case_num == 0)&(train.pn_num != 16)]

,id,case_num,pn_num,feature_num,annotation,location
13,00041_000,0,41,0,[],[]
14,00041_001,0,41,1,['MOM HAS THYROID PROBLEMS'],['532 556']
15,00041_002,0,41,2,['PRESSURE ON HER CHEST'],['263 284']
16,00041_003,0,41,3,"['COMES AND GOES', 'HAPPENED 5-6 TIMES']","['131 145', '150 168']"
17,00041_004,0,41,4,[],[]
...,...,...,...,...,...,...
1295,02436_008,0,2436,8,[],[]
1296,02436_009,0,2436,9,"['chest palpitations', 'chest palpitations']","['71 89', '109 127']"
1297,02436_010,0,2436,10,['2-3 month'],['50 59']
1298,02436_011,0,2436,11,['17 year old'],['26 37']


In [12]:
train[(train.case_num == 0)&(train.pn_num != 16)].pn_num.unique()

array([  41,   46,   82,  100,  161,  188,  211,  224,  225,  234,  331,
        344,  352,  372,  390,  426,  464,  489,  495,  510,  575,  583,
        585,  625,  653,  669,  673,  699,  701,  719,  768,  769,  777,
        796,  804,  810,  816,  817,  821,  831,  861,  883,  970,  977,
        987, 1010, 1110, 1115, 1133, 1146, 1149, 1151, 1165, 1173, 1186,
       1200, 1209, 1235, 1250, 1264, 1270, 1281, 1318, 1376, 1512, 1547,
       1581, 1587, 1613, 1622, 1652, 1692, 1694, 1699, 1723, 1742, 1809,
       1835, 1847, 1855, 1881, 1939, 1948, 2078, 2151, 2163, 2184, 2198,
       2201, 2223, 2271, 2285, 2290, 2308, 2315, 2425, 2428, 2430, 2436],
      dtype=int64)

In [13]:
train[(train.case_num == 0)&(train.pn_num != 16)]

,id,case_num,pn_num,feature_num,annotation,location
13,00041_000,0,41,0,[],[]
14,00041_001,0,41,1,['MOM HAS THYROID PROBLEMS'],['532 556']
15,00041_002,0,41,2,['PRESSURE ON HER CHEST'],['263 284']
16,00041_003,0,41,3,"['COMES AND GOES', 'HAPPENED 5-6 TIMES']","['131 145', '150 168']"
17,00041_004,0,41,4,[],[]
...,...,...,...,...,...,...
1295,02436_008,0,2436,8,[],[]
1296,02436_009,0,2436,9,"['chest palpitations', 'chest palpitations']","['71 89', '109 127']"
1297,02436_010,0,2436,10,['2-3 month'],['50 59']
1298,02436_011,0,2436,11,['17 year old'],['26 37']


In [14]:
train[(train.case_num == 0)&(train.pn_num != 16)&((train.feature_num == 0) | (train.feature_num == 1))]

,id,case_num,pn_num,feature_num,annotation,location
13,00041_000,0,41,0,[],[]
14,00041_001,0,41,1,['MOM HAS THYROID PROBLEMS'],['532 556']
26,00046_000,0,46,0,['father: heart attack'],['824 844']
27,00046_001,0,46,1,['mom: thyroid issues'],['803 822']
39,00082_000,0,82,0,['Father MI'],['622 631']
...,...,...,...,...,...,...
1262,02428_001,0,2428,1,['mother hyroid problem'],['551 557;566 580']
1274,02430_000,0,2430,0,['MI father'],['845 847;855 861']
1275,02430_001,0,2430,1,['thyroid dx mother'],['814 824;832 838']
1287,02436_000,0,2436,0,"['Father MI', 'father had a heart attack']","['915 924', '545 570']"


In [15]:
new_train = train[(train.case_num == 0)&(train.pn_num != 16)&((train.feature_num == 0) | (train.feature_num == 1))][['pn_num','annotation',"location"]]

In [16]:
new_train

,pn_num,annotation,location
13,41,[],[]
14,41,['MOM HAS THYROID PROBLEMS'],['532 556']
26,46,['father: heart attack'],['824 844']
27,46,['mom: thyroid issues'],['803 822']
39,82,['Father MI'],['622 631']
...,...,...,...
1262,2428,['mother hyroid problem'],['551 557;566 580']
1274,2430,['MI father'],['845 847;855 861']
1275,2430,['thyroid dx mother'],['814 824;832 838']
1287,2436,"['Father MI', 'father had a heart attack']","['915 924', '545 570']"


#### Removing all [ ] or null columns 

In [17]:
new_train = new_train[new_train.annotation != '[]']

In [18]:
new_train

,pn_num,annotation,location
14,41,['MOM HAS THYROID PROBLEMS'],['532 556']
26,46,['father: heart attack'],['824 844']
27,46,['mom: thyroid issues'],['803 822']
39,82,['Father MI'],['622 631']
40,82,['Mom thyroid problem'],['633 652']
...,...,...,...
1262,2428,['mother hyroid problem'],['551 557;566 580']
1274,2430,['MI father'],['845 847;855 861']
1275,2430,['thyroid dx mother'],['814 824;832 838']
1287,2436,"['Father MI', 'father had a heart attack']","['915 924', '545 570']"


#### Removing all [  &  ] in strings

In [19]:
new_train["annotation"] = new_train.annotation.str.replace('[','')
new_train["annotation"] = new_train.annotation.str.replace(']','')
new_train["location"] = new_train.location.str.replace('[','')
new_train["location"] = new_train.location.str.replace(']','')

#### special row in which data was stored incorrectly need to correct it

In [20]:
new_train.loc[new_train.location == "'883 900'",'annotation'] = "['dad  heart attack']"

In [21]:
new_train[new_train.pn_num == 1581][0:1].annotation

871    ['dad  heart attack']
Name: annotation, dtype: object

#### splitting locations and annotations 

In [24]:
new_train = (new_train.set_index(['pn_num']) 
   .apply(lambda col: col.str.split(',').explode())
   .reset_index()
   .reindex(new_train.columns, axis=1))

In [25]:
new_train.head(3)

,pn_num,annotation,location
0,41,'MOM HAS THYROID PROBLEMS','532 556'
1,46,'father: heart attack','824 844'
2,46,'mom: thyroid issues','803 822'


#### Creating two new columns "start location" and "end location"

In [26]:
new_train["start_location"] = new_train["location"].apply(lambda x: x.split()[0][1:])

In [27]:
new_train["end_location"] = new_train["location"].apply(lambda x: x.split()[-1][:-1])

In [28]:
new_train.head(3)

,pn_num,annotation,location,start_location,end_location
0,41,'MOM HAS THYROID PROBLEMS','532 556',532,556
1,46,'father: heart attack','824 844',824,844
2,46,'mom: thyroid issues','803 822',803,822


#### Finding unique patient numbers in the train dataset

In [29]:
pn_Num_list = list(new_train.pn_num.unique())

In [30]:
len(pn_Num_list)

96

#### Creating train dataset in required format
#### format - [(text,{"entities":[(start,end,label)]})]

In [31]:
new_train.loc[[12,13],]

,pn_num,annotation,location,start_location,end_location
12,211,'thyroid problem','665 680',665,680
13,211,'Mother has a thyroid problem','652 680',652,680


In [32]:
new_train.loc[[25,26],]

,pn_num,annotation,location,start_location,end_location
25,352,'father with heart attack recently','905 938',905,938
26,352,'father with heart attack','905 929',905,929


In [33]:
new_train = new_train.drop([12,26],axis=0)

In [34]:
final_train = []   # appending all the individual rows
for patient_number in pn_Num_list:  # for each unique patient number
    entities = []  # saving individaual entities locations
    for row_index,row in new_train[new_train.pn_num == patient_number].iterrows():  # for each unique patient number 
        
        entities.append((int(row["start_location"]),int(row["end_location"]),"Feature")) # store the locations in numeric format
        
    text = patient_notes[patient_notes.pn_num == patient_number]["pn_history"].values[0] # save the text for the unique patient number
    
    final_train.append((text,{"entities":entities}))  # for each unique pn_number append to final list
            
            


In [35]:
final_train[0]  # sample train

('17 Y/O M CAME TO THE CLINIC C/O HEART POUNDING. STARTED 2-3 MO AGO. IT STARTED SUDDENLY. DOES NOT RECALL ANY TRIGGERING EVENTS. IT COMES AND GOES, IT HAPPENED 5-6 TIMES SINCE IT STARTED. IT LASTS 3-4 MIN, AFTER THAN  JUST GOES AWAY. HE HAS ALSO EXPERIENCING SOB, PRESSURE ON HER CHEST WHEN HE HAS THIS ATTACK. HE IS A COLLEGE STUDENT, EXPERIENCING SOME STRESS RECENTLY. \r\nDENIES COUGH, CHEST PAIN.\r\nROS NEG EXCEPT AS ABOVE.\r\nPMH NONE. MEDS ATEROL, FOR HIS STUDIES, SHARING W HIS ROOMMATE. NKDA.\r\nPSH/ HOSP/ TRAVEL/ TRAUMA NONE.\r\nFH MOM HAS THYROID PROBLEMS.\r\nSH SEX ACTIVE W GIRLFRIEND, NO STDS, USING CONDOMS. SMOKE NONE. ETOH ONLY WEEKENDS. DRUG ONLY ONCE, 1 MO AGO.',
 {'entities': [(532, 556, 'Feature')]})

#### Saving the train data in json file

In [36]:
# Serializing json 
json_object = json.dumps(final_train, indent = 4)
  
# Writing to sample.json
with open("sample.json", "w") as outfile:
    outfile.write(json_object)

#### Loading train data from json file

In [37]:
import json
 
# Opening JSON file
f = open("sample.json")
 
# returns JSON object as
# a dictionary
data = json.load(f)

In [38]:
data[0]

['17 Y/O M CAME TO THE CLINIC C/O HEART POUNDING. STARTED 2-3 MO AGO. IT STARTED SUDDENLY. DOES NOT RECALL ANY TRIGGERING EVENTS. IT COMES AND GOES, IT HAPPENED 5-6 TIMES SINCE IT STARTED. IT LASTS 3-4 MIN, AFTER THAN  JUST GOES AWAY. HE HAS ALSO EXPERIENCING SOB, PRESSURE ON HER CHEST WHEN HE HAS THIS ATTACK. HE IS A COLLEGE STUDENT, EXPERIENCING SOME STRESS RECENTLY. \r\nDENIES COUGH, CHEST PAIN.\r\nROS NEG EXCEPT AS ABOVE.\r\nPMH NONE. MEDS ATEROL, FOR HIS STUDIES, SHARING W HIS ROOMMATE. NKDA.\r\nPSH/ HOSP/ TRAVEL/ TRAUMA NONE.\r\nFH MOM HAS THYROID PROBLEMS.\r\nSH SEX ACTIVE W GIRLFRIEND, NO STDS, USING CONDOMS. SMOKE NONE. ETOH ONLY WEEKENDS. DRUG ONLY ONCE, 1 MO AGO.',
 {'entities': [[532, 556, 'Feature']]}]

In [39]:
train = final_train

#### Further Data Preperation for spacy 3.0 version

#### Deviding the data into train and validation

In [40]:
train = data[:76]
validation = data[76:]

In [41]:
len(train),len(validation)

(76, 20)

In [42]:
from spacy.tokens import DocBin
import spacy
import json
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin
import random

In [43]:
# converting the json file to spacy file in order to use it in training model

nlp = spacy.blank("en")
def create_training(TRAIN_DATA):
    db = DocBin()
    for text, annot in tqdm(TRAIN_DATA):
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print ("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    return (db)

In [ ]:
create_training(train)

In [ ]:
# 12,26 -  dropped theses rows as, these were duplicate rows having similar deatures

In [44]:
#train data set

camp_train = create_training(train)
camp_train.to_disk("nbme_train.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 76/76 [00:00<00:00, 584.62it/s]


In [45]:
#test data set

camp_train = create_training(validation)
camp_train.to_disk("nbme_validation.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 623.51it/s]


In [48]:
!python -m spacy init fill-config ./base_config.cfg ./config.cfg

[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [49]:
!python -m spacy train config.cfg --output ./output

[+] Created output directory: output
[i] Saving to output directory: output
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     74.83    0.00    0.00    0.00    0.00
  2     200        170.93   1801.64   80.00   78.95   81.08    0.80
  5     400         52.79    112.59   80.00   78.95   81.08    0.80
  7     600         37.09     48.84   85.71   82.50   89.19    0.86
 10     800        113.19     38.36   83.78   83.78   83.78    0.84
 13    1000         29.05     18.67   81.58   79.49   83.78    0.82
 15    1200         42.24     14.46   86.84   84.62   89.19    0.87
 18    1400         17.45      8.03   80.52   77.50 

[2022-03-16 11:48:29,526] [INFO] Set up nlp object from config
[2022-03-16 11:48:29,534] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-03-16 11:48:29,537] [INFO] Created vocabulary
[2022-03-16 11:48:29,537] [INFO] Finished initializing nlp object
[2022-03-16 11:48:30,232] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


#### Test data

In [77]:
nlp = spacy.load("output/model-best")
doc = nlp(test)

In [81]:
test = np.array(patient_notes.loc[patient_notes.pn_num == 16,"pn_history"])[0]

In [80]:
for ent in doc.ents:
    print(ent.text, ent.label_,ent.start_char, ent.end_char)

mom with "thyroid disease Feature 668 693
dad with recent heart attcak Feature 696 724


In [79]:
train_data = pd.read_csv("train.csv")
train_data[train_data.pn_num == 16]

,id,case_num,pn_num,feature_num,annotation,location
0,00016_000,0,16,0,['dad with recent heart attcak'],['696 724']
1,00016_001,0,16,1,"['mom with ""thyroid disease']",['668 693']
2,00016_002,0,16,2,['chest pressure'],['203 217']
3,00016_003,0,16,3,"['intermittent episodes', 'episode']","['70 91', '176 183']"
4,00016_004,0,16,4,['felt as if he were going to pass out'],['222 258']
5,00016_005,0,16,5,[],[]
6,00016_006,0,16,6,"['adderall', 'adderrall', 'adderrall']","['321 329', '404 413', '652 661']"
7,00016_007,0,16,7,[],[]
8,00016_008,0,16,8,[],[]
9,00016_009,0,16,9,"['palpitations', 'heart beating/pounding']","['26 38', '96 118']"
